In [1]:
### DOWNLOAD ROME GRAPH DATASET TO A FOLDER CALLED ROME ###
# DOWNLOAD PAGE: http://www.graphdrawing.org/data.html
# DOWNLOAD URL: http://www.graphdrawing.org/data/rome-graphml.tgz

### UNCOMMENT TO DOWNLOAD ROME DATASET IN FOLDER rome IF IT IS NOT ALREADY DOWNLOADED 
"""
import os
import requests
import tarfile
from io import BytesIO

URL = "http://www.graphdrawing.org/data/rome-graphml.tgz"
TARGET_DIR = "rome"

def download_and_extract_graphs(url, target_dir):
    if not os.path.isdir(target_dir):
        print("Downloading dataset...")
        response = requests.get(url, stream=True)
        response.raise_for_status()

        fileobj = BytesIO(response.content)
        with tarfile.open(fileobj=fileobj, mode="r:gz") as tar:
            tar.extractall(path=".")
        
        print("Extraction completed.")
    else:
        print(f"'{target_dir}' already exists — skipping download.")

download_and_extract_graphs(URL, TARGET_DIR)
"""

'\nimport os\nimport requests\nimport tarfile\nfrom io import BytesIO\n\nURL = "http://www.graphdrawing.org/data/rome-graphml.tgz"\nTARGET_DIR = "rome"\n\ndef download_and_extract_graphs(url, target_dir):\n    if not os.path.isdir(target_dir):\n        print("Downloading dataset...")\n        response = requests.get(url, stream=True)\n        response.raise_for_status()\n\n        fileobj = BytesIO(response.content)\n        with tarfile.open(fileobj=fileobj, mode="r:gz") as tar:\n            tar.extractall(path=".")\n        \n        print("Extraction completed.")\n    else:\n        print(f"\'{target_dir}\' already exists — skipping download.")\n\ndownload_and_extract_graphs(URL, TARGET_DIR)\n'

In [2]:
import sys
sys.path.append('./paralel_planarity_criterion')

import os
import random
import networkx as nx
from glob import glob

from planarity_criterion import PlanarityCriterion  
import planar_printer

In [3]:
def load_random_graphs_from_folder(
    folder_path,
    node_range=None,
    cycle_metric_range=None,
    num_graphs=None,
    seed=42,
    verbose=1
):
    """
    Load a random subset (or all) of undirected graphs from .graphml files in a folder,
    filtered by node count (extracted from filename) and cycle space metric.

    Parameters:
        folder_path (str): Directory containing .graphml files.
        node_range (list or None): [min_nodes, max_nodes] constraint, or None to ignore.
        cycle_metric_range (list or None): [min, max] for (cycle_space_dim * num_edges), or None to ignore.
        num_graphs (int or None): Number of graphs to load. If None, load all matching graphs.
        seed (int): Random seed for reproducibility.
        verbose (int): 0 = silent, 1 = summary only, 2 = print stats for each selected graph.

    Returns:
        List of (filename, graph) tuples.
    """
    import os
    import re
    import random
    import networkx as nx
    from glob import glob

    random.seed(seed)
    all_files = glob(os.path.join(folder_path, "*.graphml"))
    if verbose >= 1:
        print(f"Found {len(all_files)} .graphml files in folder.")

    eligible_graphs = []

    for file in all_files:
        try:
            # Extract node count from filename
            base = os.path.basename(file)
            match = re.findall(r"\.(\d+)\.graphml$", base)
            if not match:
                continue
            file_node_count = int(match[0])
            
            # Node constraint check
            if node_range:
                min_nodes, max_nodes = node_range
                if not (min_nodes <= file_node_count <= max_nodes):
                    continue

            # Load graph
            G = nx.read_graphml(file)
            G = nx.Graph(G)
            n = G.number_of_nodes()
            m = G.number_of_edges()
            cycle_space_dim = m - n + 1
            cycle_metric = cycle_space_dim * m

            # Cycle metric constraint check
            if cycle_metric_range:
                min_cyc, max_cyc = cycle_metric_range
                if not (min_cyc <= cycle_metric <= max_cyc):
                    continue

            eligible_graphs.append((file, G))

        except Exception as e:
            if verbose >= 1:
                print(f"⚠️ Error reading {file}: {e}")

    if verbose >= 1:
        print(f"{len(eligible_graphs)} files matched the constraints.")

    if num_graphs is not None:
        selected_graphs = random.sample(eligible_graphs, min(num_graphs, len(eligible_graphs)))
        if verbose >= 1:
            print(f"Selected {len(selected_graphs)} graphs at random.\n")
    else:
        selected_graphs = eligible_graphs
        if verbose >= 1:
            print(f"Selected ALL {len(selected_graphs)} eligible graphs.\n")

    if verbose == 2:
        for file, G in selected_graphs:
            n = G.number_of_nodes()
            m = G.number_of_edges()
            cycle_space_dim = m - n + 1
            cycle_metric = cycle_space_dim * m
            print(f"{os.path.basename(file)} → nodes: {n}, edges: {m}, cycle_space_dim: {cycle_space_dim}, cycle_metric: {cycle_metric}")

    return selected_graphs


In [4]:
import time
import numpy as np


import time
import numpy as np

def test_planarity_on_graphs(graphs, verbose=2, algorithm="Planar-1"):
    """
    Test planarity on a list of (filename, graph) tuples using one of two algorithms.

    Parameters:
        graphs: List of (filename, networkx.Graph) tuples.
        verbose (int): 
            0 = silent unless mismatch,
            1 = summary only,
            2 = detailed output with timing, cycles, CNF vars.
        algorithm (str): 'Planar-1' (default, solver.is_planar) or 'Planar-2' (get_embedding method).

    Returns:
        timings: List of tuples (filename, your_time, networkx_time, ratio)
    """
    import os
    import networkx as nx
    from planarity_criterion import PlanarityCriterion
    import planar_printer

    solver = PlanarityCriterion()
    failed_graphs = []
    timings = []

    for file, G in graphs:
        try:
            n = G.number_of_nodes()
            m = G.number_of_edges()
            cycle_space_dim = m - n + 1
            cnf_vars = m * cycle_space_dim

            
            start_my = time.time()
            if algorithm == "Planar-1":
                my_result, info = solver.is_planar(G)
            elif algorithm == "Planar-2":
                try:
                    planar, _ = planar_printer.get_embbeding(G)
                    my_result = planar t
                    info = {"source": "Planar-2"}
                except Exception as e:
                    my_result = False
                    info = {"error": str(e)}
            else:
                raise ValueError(f"Unknown algorithm: {algorithm}")
            end_my = time.time()

            # Time NetworkX
            start_nx = time.time()
            nx_result, _ = nx.check_planarity(G)
            end_nx = time.time()

            my_time = end_my - start_my
            nx_time = end_nx - start_nx
            ratio = my_time / (nx_time + 1e-9)
            timings.append((os.path.basename(file), my_time, nx_time, ratio))

            if verbose == 2:
                print(f"Testing {os.path.basename(file)}")
                print(f" - Nodes: {n}, Edges: {m}")
                print(f" - Cycle space dimension: {cycle_space_dim}")
                print(f" - CNF variables (edges × cycle_space_dim): {cnf_vars}")
                print(f" - ⏱️ {algorithm} time: {my_time:.6f}s, NetworkX: {nx_time:.6f}s, Ratio: {ratio:.2f}x")

            if my_result != nx_result:
                failed_graphs.append(file)
                if verbose >= 1:
                    print("❌ Mismatch!")
                    print("File:", file)
                    if verbose == 2:
                        print("Nodes:", G.nodes())
                        print("Edges:", G.edges())
                        print("Info:", info)
                print()
            elif verbose == 2:
                print("✅ Match\n")

        except Exception as e:
            if verbose >= 1:
                print(f"⚠️ Error processing {file}: {e}")
            failed_graphs.append(file)

    # Summary
    if verbose >= 1 or failed_graphs:
        print(f"\n✅ Done. {len(failed_graphs)} mismatches found.")
        if failed_graphs:
            print("❌ Mismatched files:")
            for f in failed_graphs:
                print(" -", f)
        else:
            print("🎉 All tests passed.")

    if verbose >= 1:
        my_times = np.array([t[1] for t in timings])
        nx_times = np.array([t[2] for t in timings])
        ratios = np.array([t[3] for t in timings])

        print("\nTiming Summary:")
        print(f"Mean time — {algorithm}: {np.mean(my_times):.6f}s")
        print(f"Mean time — NetworkX   : {np.mean(nx_times):.6f}s")
        print(f"Mean time ratio ({algorithm} / networkx): {np.mean(ratios):.2f}x")

        if verbose == 2:
            print("\nPer-graph Timing & Ratios:")
            for fname, my_t, nx_t, r in timings:
                print(f"{fname}: {algorithm} = {my_t:.6f}s, NetworkX = {nx_t:.6f}s, Ratio = {r:.2f}x")

    return timings

SyntaxError: invalid syntax (3743619815.py, line 46)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(graphs, cols=4, node_size=100):
    """
    Display all graphs in the list using matplotlib.

    Parameters:
        graphs: List of (filename, networkx.Graph) tuples.
        cols: Number of columns in the subplot grid.
        node_size: Size of graph nodes.
    """
    total = len(graphs)
    rows = (total + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    axes = axes.flatten() if total > 1 else [axes]

    for i, (filename, G) in enumerate(graphs):
        ax = axes[i]
        # ax.set_title(os.path.basename(filename), fontsize=8)
        nx.draw(G, ax=ax, with_labels=False, node_size=node_size)
    
    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:

"""
Mejores marcas:

Testing grafo4453.57.graphml
 - Nodes: 57, Edges: 72
 - Cycle space dimension: 16
 - CNF variables (edges × cycle_space_dim): 1152
 - ⏱️ Planar-1 time: 588.986076s, NetworkX: 0.001049s, Ratio: 561706.30x
 ✅ Match

 Testing grafo3181.55.graphml
 - Nodes: 55, Edges: 77
 - Cycle space dimension: 23
 - CNF variables (edges × cycle_space_dim): 1771
 - ⏱️ Planar-1 time: 33745.582342s, NetworkX: 0.001993s, Ratio: 16930521.53x
✅ Match

 
"""

graphs = load_random_graphs_from_folder("rome", node_range=[5, 15], cycle_metric_range=None, num_graphs=20, seed=123)  
plot_graphs(graphs)
timings = test_planarity_on_graphs(graphs, algorithm="Planar-1")